Ahmet Alperen Gungor, 28847
Elifnur Ozturk, 28857 
Ayse Sena Acar, 29506
Baris Bakirdoven, 29028

In [ ]:
from keras.datasets import cifar10
from sklearn.metrics import accuracy_score
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount("./drive")

Mounted at ./drive


In [ ]:
# Function to load each batch file
def load_cifar_batch(file):
    with open("/content/drive/MyDrive/dataset/" + file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict[b'data'], dict[b'labels']

# Define batch files
batch_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Grid of hyperparameters to search over
max_depth_values = [10, 20, 30, None]
n_estimators_values = [50, 100, 150, 200]

# Store the best parameters and their corresponding accuracy
best_params = None
best_accuracy = 0

In [ ]:
# Iterate over all possible combinations of hyperparameters
for max_depth in max_depth_values:
    for n_estimators in n_estimators_values:
        fold_accuracies = []
        
        # Perform 5-fold cross-validation
        for i in range(len(batch_files)):
            # Create a new list without the i-th batch file
            train_files = batch_files[:i] + batch_files[i+1:]
            
            # Load, reshape and combine training data batches
            X_train = np.concatenate([load_cifar_batch(batch)[0].reshape(-1, 3 * 32 * 32) / 255.0 for batch in train_files])
            y_train = np.concatenate([load_cifar_batch(batch)[1] for batch in train_files])

            # Load and reshape validation data batch
            X_val = load_cifar_batch(batch_files[i])[0].reshape(-1, 3 * 32 * 32) / 255.0
            y_val = load_cifar_batch(batch_files[i])[1]

            # Train a Random Forest model
            rf = RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators)
            rf.fit(X_train, y_train)

            # Validate the model
            val_preds = rf.predict(X_val)
            val_accuracy = accuracy_score(y_val, val_preds)

            fold_accuracies.append(val_accuracy)
        
        # Compute the mean accuracy over the 5 folds
        mean_accuracy = np.mean(fold_accuracies)
        
        # Update the best parameters if this configuration is better
        if mean_accuracy > best_accuracy:
            best_accuracy = mean_accuracy
            best_params = (max_depth, n_estimators)
        
        print(f"max_depth = {max_depth}, n_estimators = {n_estimators}, Validation Accuracy = {mean_accuracy}")

max_depth = 10, n_estimators = 50, Validation Accuracy = 0.4143
max_depth = 10, n_estimators = 100, Validation Accuracy = 0.42013999999999996
max_depth = 10, n_estimators = 150, Validation Accuracy = 0.42282000000000003
max_depth = 10, n_estimators = 200, Validation Accuracy = 0.42338000000000003
max_depth = 20, n_estimators = 50, Validation Accuracy = 0.43796
max_depth = 20, n_estimators = 100, Validation Accuracy = 0.4551
max_depth = 20, n_estimators = 150, Validation Accuracy = 0.46475999999999995
max_depth = 20, n_estimators = 200, Validation Accuracy = 0.46878000000000003
max_depth = 30, n_estimators = 50, Validation Accuracy = 0.43548
max_depth = 30, n_estimators = 100, Validation Accuracy = 0.4562799999999999


KeyboardInterrupt: ignored

In [ ]:

print(f"Best hyperparameters: max_depth = {best_params[0]}, n_estimators = {best_params[1]}, Cross-Validation Accuracy = {best_accuracy}")

# Now, use the best hyperparameters to train a model on all the training data
X_train_combined = np.concatenate([load_cifar_batch(batch)[0].reshape(-1, 3 * 32 * 32) / 255.0 for batch in batch_files])
y_train_combined = np.concatenate([load_cifar_batch(batch)[1] for batch in batch_files])

rf_final = RandomForestClassifier(max_depth=best_params[0], n_estimators=best_params[1])
rf_final.fit(X_train_combined, y_train_combined)

# Load the test batch
test_batch = load_cifar_batch('test_batch')

# reshape test data and labels
X_test = test_batch[0].reshape(-1, 3 * 32 * 32) / 255.0
y_test = test_batch[1]

# Test the final model
test_preds = rf_final.predict(X_test)
test_accuracy = accuracy_score(y_test, test_preds)

print(f"Test Accuracy: {test_accuracy}")

Best hyperparameters: max_depth = 20, n_estimators = 200, Cross-Validation Accuracy = 0.46878000000000003


KeyboardInterrupt: ignored